# Robust OpenCode Execution with Error Handling

This notebook provides comprehensive error handling and retry logic for OpenCode operations,
including automatic retries, fallback strategies, and graceful degradation.

**Parameters:**
- `prompt`: The prompt to execute with error handling
- `model`: Primary model to use
- `config_file`: Error handling configuration file
- `max_retries`: Maximum retry attempts (overrides config)
- `timeout`: Operation timeout in seconds

In [ ]:
# Default parameters - will be overridden by papermill
prompt = "Test prompt for error handling"
model = ""
config_file = "error_config.yaml"
max_retries = ""
timeout = "300"
verbose = "true"

In [ ]:
#!/bin/bash

# Setup error handling environment
echo "=== Robust OpenCode Execution with Error Handling ==="
echo "Prompt: $prompt"
echo "Primary model: ${model:-default}"
echo "Config file: $config_file"
echo "Max retries: ${max_retries:-from config}"
echo "Timeout: ${timeout}s"
echo "Verbose: $verbose"
echo

# Create error handling directory
TIMESTAMP=$(date '+%Y%m%d_%H%M%S')
ERROR_DIR="error_logs/${TIMESTAMP}"
mkdir -p "$ERROR_DIR"

# Error handling files
ERROR_LOG="$ERROR_DIR/execution.log"
RETRY_LOG="$ERROR_DIR/retry.log"
OUTPUT_FILE="$ERROR_DIR/result.md"
STATUS_FILE="$ERROR_DIR/status.json"

echo "Error directory: $ERROR_DIR"
echo "Error log: $ERROR_LOG"
echo "Output file: $OUTPUT_FILE"
echo

# Initialize status tracking
cat > "$STATUS_FILE" << EOF
{
  "timestamp": "$(date -Iseconds)",
  "prompt": "$prompt",
  "primary_model": "${model:-default}",
  "max_retries": "${max_retries:-from config}",
  "timeout": "$timeout",
  "status": "initializing",
  "attempts": [],
  "final_result": null
}
EOF

echo "✅ Error handling environment initialized"

In [ ]:
#!/bin/bash

# Load error handling configuration
echo "=== Loading Error Handling Configuration ==="

if [ ! -f "$config_file" ]; then
    echo "⚠️  Warning: Config file '$config_file' not found, using defaults"
    
    # Set default values
    CONFIG_MAX_RETRIES=3
    CONFIG_RETRY_DELAY=5
    CONFIG_BACKOFF_MULTIPLIER=2
    CONFIG_MAX_RETRY_DELAY=60
    FALLBACK_MODELS=("claude-3-5-sonnet-20241022" "claude-3-haiku-20240307")
else
    echo "📋 Loading configuration from: $config_file"
    
    # Extract key configuration values (basic YAML parsing)
    CONFIG_MAX_RETRIES=$(grep -A 20 "^error_handling:" "$config_file" | grep "max_retries:" | head -1 | sed 's/.*max_retries: *//' || echo "3")
    CONFIG_RETRY_DELAY=$(grep -A 20 "^error_handling:" "$config_file" | grep "retry_delay:" | head -1 | sed 's/.*retry_delay: *//' || echo "5")
    CONFIG_BACKOFF_MULTIPLIER=$(grep -A 20 "^error_handling:" "$config_file" | grep "backoff_multiplier:" | head -1 | sed 's/.*backoff_multiplier: *//' || echo "2")
    CONFIG_MAX_RETRY_DELAY=$(grep -A 20 "^error_handling:" "$config_file" | grep "max_retry_delay:" | head -1 | sed 's/.*max_retry_delay: *//' || echo "60")
    
    # Extract fallback models
    FALLBACK_MODELS=($(grep -A 10 "fallback_models:" "$config_file" | grep "^\s*-" | sed 's/.*- *"\?//' | sed 's/"\?$//' | tr '\n' ' '))
    
    echo "✅ Configuration loaded successfully"
fi

# Override with parameter if provided
if [ -n "$max_retries" ]; then
    CONFIG_MAX_RETRIES="$max_retries"
fi

echo "Configuration:"
echo "  Max retries: $CONFIG_MAX_RETRIES"
echo "  Base retry delay: ${CONFIG_RETRY_DELAY}s"
echo "  Backoff multiplier: $CONFIG_BACKOFF_MULTIPLIER"
echo "  Max retry delay: ${CONFIG_MAX_RETRY_DELAY}s"
echo "  Fallback models: ${FALLBACK_MODELS[*]:-none}"
echo

# Initialize retry tracking
ATTEMPT_COUNT=0
CURRENT_DELAY="$CONFIG_RETRY_DELAY"

In [ ]:
#!/bin/bash

# Define error detection and handling functions
echo "=== Setting up Error Detection Functions ==="

# Function to classify error type
classify_error() {
    local error_output="$1"
    
    # Network errors
    if echo "$error_output" | grep -qi "connection\|network\|dns"; then
        echo "network"
        return 0
    fi
    
    # Authentication errors
    if echo "$error_output" | grep -qi "auth\|unauthorized\|access denied\|api key"; then
        echo "auth"
        return 0
    fi
    
    # Rate limit errors
    if echo "$error_output" | grep -qi "rate limit\|too many requests\|quota"; then
        echo "rate_limit"
        return 0
    fi
    
    # Timeout errors
    if echo "$error_output" | grep -qi "timeout\|deadline"; then
        echo "timeout"
        return 0
    fi
    
    # Model/API errors
    if echo "$error_output" | grep -qi "model.*not found\|service unavailable\|internal server"; then
        echo "api"
        return 0
    fi
    
    # Validation errors
    if echo "$error_output" | grep -qi "invalid.*prompt\|prompt.*too long\|malformed"; then
        echo "validation"
        return 0
    fi
    
    # Unknown error
    echo "unknown"
}

# Function to determine if error is retryable
is_retryable() {
    local error_type="$1"
    
    case "$error_type" in
        "network"|"rate_limit"|"api"|"timeout"|"unknown")
            return 0  # Retryable
            ;;
        "auth"|"validation")
            return 1  # Not retryable
            ;;
        *)
            return 0  # Default to retryable
            ;;
    esac
}

# Function to calculate retry delay with exponential backoff
calculate_delay() {
    local attempt="$1"
    local base_delay="$2"
    local multiplier="$3"
    local max_delay="$4"
    
    local calculated_delay=$((base_delay * multiplier ** (attempt - 1)))
    
    if [ $calculated_delay -gt $max_delay ]; then
        echo $max_delay
    else
        echo $calculated_delay
    fi
}

echo "✅ Error detection functions defined"

In [ ]:
#!/bin/bash

# Main robust execution loop with retry logic
echo "=== Starting Robust Execution Loop ==="

SUCCESS=false
CURRENT_MODEL="$model"
FALLBACK_INDEX=0

# Update status to running
if command -v jq >/dev/null 2>&1; then
    jq '.status = "running"' "$STATUS_FILE" > "${STATUS_FILE}.tmp" && mv "${STATUS_FILE}.tmp" "$STATUS_FILE"
fi

while [ $ATTEMPT_COUNT -lt $CONFIG_MAX_RETRIES ] && [ "$SUCCESS" = false ]; do
    ATTEMPT_COUNT=$((ATTEMPT_COUNT + 1))
    
    echo "--- Attempt $ATTEMPT_COUNT/$CONFIG_MAX_RETRIES ---"
    echo "Model: ${CURRENT_MODEL:-default}"
    echo "Timeout: ${timeout}s"
    
    # Log attempt start
    ATTEMPT_START=$(date -Iseconds)
    echo "[$ATTEMPT_START] Attempt $ATTEMPT_COUNT started with model '${CURRENT_MODEL:-default}'" >> "$ERROR_LOG"
    
    # Execute OpenCode with timeout
    TEMP_OUTPUT="$ERROR_DIR/attempt_${ATTEMPT_COUNT}.md"
    
    if [ -n "$CURRENT_MODEL" ]; then
        timeout "$timeout" bash -c "echo '$prompt' | opencode --model '$CURRENT_MODEL'" > "$TEMP_OUTPUT" 2>&1
    else
        timeout "$timeout" bash -c "echo '$prompt' | opencode" > "$TEMP_OUTPUT" 2>&1
    fi
    
    EXIT_CODE=$?
    ATTEMPT_END=$(date -Iseconds)
    
    if [ $EXIT_CODE -eq 0 ]; then
        echo "✅ Attempt $ATTEMPT_COUNT successful!"
        SUCCESS=true
        cp "$TEMP_OUTPUT" "$OUTPUT_FILE"
        
        # Log success
        echo "[$ATTEMPT_END] Attempt $ATTEMPT_COUNT succeeded" >> "$ERROR_LOG"
        echo "SUCCESS: Attempt $ATTEMPT_COUNT succeeded with model '${CURRENT_MODEL:-default}'" >> "$RETRY_LOG"
        
        break
    else
        # Handle error
        ERROR_OUTPUT=$(cat "$TEMP_OUTPUT")
        ERROR_TYPE=$(classify_error "$ERROR_OUTPUT")
        
        echo "❌ Attempt $ATTEMPT_COUNT failed (exit code: $EXIT_CODE)"
        echo "Error type: $ERROR_TYPE"
        
        # Log error details
        echo "[$ATTEMPT_END] Attempt $ATTEMPT_COUNT failed - Type: $ERROR_TYPE, Exit: $EXIT_CODE" >> "$ERROR_LOG"
        echo "Error output: $ERROR_OUTPUT" >> "$ERROR_LOG"
        echo "ERROR: Attempt $ATTEMPT_COUNT failed - $ERROR_TYPE" >> "$RETRY_LOG"
        
        # Check if error is retryable
        if ! is_retryable "$ERROR_TYPE"; then
            echo "💥 Error type '$ERROR_TYPE' is not retryable. Aborting."
            echo "ABORT: Non-retryable error type '$ERROR_TYPE'" >> "$RETRY_LOG"
            break
        fi
        
        # Try fallback model if available and this was a model-related error
        if [ "$ERROR_TYPE" = "api" ] && [ $FALLBACK_INDEX -lt ${#FALLBACK_MODELS[@]} ] && [ ${#FALLBACK_MODELS[@]} -gt 0 ]; then
            CURRENT_MODEL="${FALLBACK_MODELS[$FALLBACK_INDEX]}"
            FALLBACK_INDEX=$((FALLBACK_INDEX + 1))
            echo "🔄 Switching to fallback model: $CURRENT_MODEL"
            echo "FALLBACK: Switching to model '$CURRENT_MODEL'" >> "$RETRY_LOG"
        fi
        
        # Calculate retry delay if we have more attempts
        if [ $ATTEMPT_COUNT -lt $CONFIG_MAX_RETRIES ]; then
            CURRENT_DELAY=$(calculate_delay $ATTEMPT_COUNT $CONFIG_RETRY_DELAY $CONFIG_BACKOFF_MULTIPLIER $CONFIG_MAX_RETRY_DELAY)
            
            echo "⏳ Waiting ${CURRENT_DELAY}s before retry..."
            echo "RETRY_DELAY: ${CURRENT_DELAY}s for attempt $((ATTEMPT_COUNT + 1))" >> "$RETRY_LOG"
            
            sleep "$CURRENT_DELAY"
        fi
    fi
    
    echo
done

echo "=== Execution Loop Completed ==="

In [ ]:
#!/bin/bash

# Handle final result and cleanup
echo "=== Processing Final Result ==="

FINAL_TIMESTAMP=$(date -Iseconds)

if [ "$SUCCESS" = true ]; then
    echo "🎉 OpenCode execution completed successfully!"
    echo "✅ Attempts: $ATTEMPT_COUNT/$CONFIG_MAX_RETRIES"
    echo "📄 Result saved to: $OUTPUT_FILE"
    
    # Update status file
    if command -v jq >/dev/null 2>&1; then
        jq --arg timestamp "$FINAL_TIMESTAMP" --arg attempts "$ATTEMPT_COUNT" --arg model "${CURRENT_MODEL:-default}" \
           '.status = "success" | .completed_at = $timestamp | .attempts_used = ($attempts | tonumber) | .successful_model = $model' \
           "$STATUS_FILE" > "${STATUS_FILE}.tmp" && mv "${STATUS_FILE}.tmp" "$STATUS_FILE"
    fi
    
    echo
    echo "--- Result Preview ---"
    head -20 "$OUTPUT_FILE"
    echo "--- End Preview ---"
    
    FINAL_STATUS="SUCCESS"
else
    echo "💥 OpenCode execution failed after $ATTEMPT_COUNT attempts"
    echo "❌ All retry attempts exhausted"
    
    # Update status file  
    if command -v jq >/dev/null 2>&1; then
        jq --arg timestamp "$FINAL_TIMESTAMP" --arg attempts "$ATTEMPT_COUNT" \
           '.status = "failed" | .failed_at = $timestamp | .attempts_used = ($attempts | tonumber)' \
           "$STATUS_FILE" > "${STATUS_FILE}.tmp" && mv "${STATUS_FILE}.tmp" "$STATUS_FILE"
    fi
    
    echo "📋 Check error logs for details:"
    echo "   Error log: $ERROR_LOG"
    echo "   Retry log: $RETRY_LOG"
    
    FINAL_STATUS="FAILED"
fi

echo
echo "=== Execution Summary ==="
echo "Status: $FINAL_STATUS"
echo "Total attempts: $ATTEMPT_COUNT/$CONFIG_MAX_RETRIES"
echo "Final model: ${CURRENT_MODEL:-default}"
echo "Error directory: $ERROR_DIR"
echo "Status file: $STATUS_FILE"
echo

# Show retry log summary
if [ -f "$RETRY_LOG" ]; then
    echo "Retry log summary:"
    cat "$RETRY_LOG"
    echo
fi

# Show final status
echo "Final status:"
cat "$STATUS_FILE"
echo

# Exit with appropriate code
if [ "$SUCCESS" = true ]; then
    echo "✅ Robust OpenCode execution completed successfully!"
    exit 0
else
    echo "❌ Robust OpenCode execution failed after all retry attempts"
    exit 1
fi